<h1>GRUPO ZAP: DATA SCIENCE CHALLENGE</h1>

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import zap_helpers as hp

RAW_PATH   = '../data-raw/'
DATA_PATH  = '../data-sample/'

GET_PARSE_SAVE_DATA = False

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

In [2]:
if GET_PARSE_SAVE_DATA:  
    # GET
    base_url  = 'https://s3.amazonaws.com/grupozap-data-challenge/data/'
    url_train = base_url + 'source-4-ds-train.json.zip'
    url_test  = base_url + 'source-4-ds-test.json.zip'

    hp.download_json(url_train, RAW_PATH, 'train-raw')
    hp.download_json(url_test, RAW_PATH, 'test-raw')
    
    # PARSE
    json_train = RAW_PATH + 'train-raw.json'
    json_test  = RAW_PATH + 'test-raw.json'

    df_train = hp.parse_json(json_train)
    df_test  = hp.parse_json(json_test)
    
    # SAVE
    train_data = DATA_PATH + 'train.pkl'
    test_data  = DATA_PATH + 'test.pkl'
    
    hp.pkl_io(train_data, method='dump', file=df_train)
    hp.pkl_io(test_data, method='dump', file=df_test)

In [ ]:
train_data = DATA_PATH + 'train.pkl'
test_data  = DATA_PATH + 'test.pkl'

df_train = hp.pkl_io(train_data)
df_test  = hp.pkl_io(test_data)

# ANÁLISE EXPLORATÓRIA

In [5]:
df_train.head()

,address_city,address_country,address_district,address_geoLocation_location_lat,address_geoLocation_location_lon,address_geoLocation_precision,address_locationId,address_neighborhood,address_state,address_street,address_streetNumber,address_unitNumber,address_zipCode,address_zone,bathrooms,bedrooms,createdAt,description,id,images,listingStatus,owner,parkingSpaces,pricingInfos_businessType,pricingInfos_price,publicationType,publisherId,suites,title,totalAreas,unitTypes,updatedAt,usableAreas,pricingInfos_monthlyCondoFee,pricingInfos_period,pricingInfos_rentalTotalPrice,pricingInfos_yearlyIptu
0,São Paulo,BR,,-23.612923,-46.614222,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da...,Jardim da Saúde,São Paulo,Rua Juvenal Galeno,53,,04290030,Zona Sul,3,4,2017-02-07T13:21:40Z,"04 dorms sendo 01 suíte e closet, sala de esta...",787c7bd19d,[https://s3-sa-east-1.amazonaws.com/vr.images....,ACTIVE,False,6,SALE,700000,STANDARD,f4603b2b52,1,PRÓXIMO A AVENIDA PRESIDENTE TANCREDO NEVES,388,TWO_STORY_HOUSE,2018-12-06T19:27:12.623Z,388,NaN,NaN,NaN,NaN
1,São Paulo,BR,,-23.643962,-46.593475,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Vila Liviero,Vila Santa Teresa (Zona Sul),São Paulo,Rua Juruaba,16,,04187320,Zona Sul,2,3,2016-03-21T18:35:17Z,"03 dorms sendo 01 suíte, sala, sala de jantar,...",4d68c0cdbe,[https://s3-sa-east-1.amazonaws.com/vr.images....,ACTIVE,False,2,SALE,336000,STANDARD,f4603b2b52,1,PRÓXIMO A FACULDADE UNIP CAMPUS ANCHIETA,129,HOME,2018-12-12T13:17:23.547Z,129,NaN,NaN,NaN,NaN
2,São Paulo,,,-23.568559,-46.647452,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Centro>Cerqueira C...,Bela Vista,São Paulo,Avenida Paulista,402,,01311000,,4,0,2018-12-18T23:47:03.425Z,"Andar com 395,70m² de área útil, 04 wcs, 05 va...",e7e0b554ac,[http://static.nidoimovel.com.br/d3d9446802a44...,ACTIVE,False,5,RENTAL,24929,STANDARD,501f6d5e94,0,Excelente Conjunto Comercial na Av. Paulista,None,COMMERCIAL_PROPERTY,NaN,396,4900.0,MONTHLY,29829.0,4040.0
3,São Paulo,,,-23.598023,-46.682781,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Vila Olimpia,Vila Olímpia,São Paulo,Rua Alvorada,1190,,04550004,,2,3,2018-10-26T16:18:28.915Z,"Viva melhor e de um jeito novo, com privilégio...",6654d93423,[https://vr-prod-vivapro-images.s3.amazonaws.c...,ACTIVE,False,2,SALE,739643,STANDARD,527c7b0cc0,1,"Apartamento com 80m², 2 vagas próximo ao Parqu...",80,APARTMENT,2018-11-08T16:38:35.434Z,80,686.0,NaN,NaN,1610.0
4,São Paulo,,,-23.582497,-46.653027,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Paraiso,Paraíso,São Paulo,Rua Curitiba,380,,04005030,,5,4,2018-12-14T18:06:51.342Z,Sua área é ocupada majoritariamente por instit...,9ffaf676ae,[https://vr-prod-vivapro-images.s3.amazonaws.c...,ACTIVE,False,5,SALE,7520099,STANDARD,527c7b0cc0,4,"Apartamento 332m², 4 suítes à venda próximo ao...",332,APARTMENT,2018-12-14T18:06:55.140Z,3322,6230.0,NaN,NaN,18900.0


In [6]:
df_train.dtypes

address_city                         object
address_country                      object
address_district                     object
address_geoLocation_location_lat    float64
address_geoLocation_location_lon    float64
address_geoLocation_precision        object
address_locationId                   object
address_neighborhood                 object
address_state                        object
address_street                       object
address_streetNumber                 object
address_unitNumber                   object
address_zipCode                      object
address_zone                         object
bathrooms                            object
bedrooms                             object
createdAt                            object
description                          object
id                                   object
images                               object
listingStatus                        object
owner                                  bool
parkingSpaces                   

#### Removendo a coluna *images*, composta por listas com URLs para imagens do imóvel. Não trabalharemos com imagem nesse modelo.

In [7]:
columns_to_remove = ['images']
df_train.drop(axis=1, columns=columns_to_remove, inplace=True)

In [14]:
distinct_count = df_train.select_dtypes(include=['object']).apply(lambda x: x.value_counts().shape[0])
print(distinct_count)

address_city                          1
address_country                       2
address_district                      3
address_geoLocation_precision         5
address_locationId                 1185
address_neighborhood               1634
address_state                         4
address_street                    12933
address_streetNumber               3357
address_unitNumber                   29
address_zipCode                   15925
address_zone                          6
bathrooms                            56
bedrooms                             30
createdAt                        132239
description                      122879
id                               133964
listingStatus                         1
parkingSpaces                       119
pricingInfos_businessType             2
publicationType                       2
publisherId                        3533
suites                               22
title                             89500
totalAreas                         1975


#### Algumas variáveis categóricas possuem poucos valores distintos. Vamos analisar esses casos: 

In [27]:
df_train.shape

(133964, 36)

In [24]:
df_train.loc[:, list(distinct_count[distinct_count <= 6].index)].apply(lambda x: x.value_counts(dropna=False)).T.stack()

address_city                   São Paulo             133964.0
address_country                                      120112.0
                               BR                     13852.0
address_district                                     133951.0
                               Jaraguá                   11.0
                               Jaçanã                     2.0
address_geoLocation_precision  APPROXIMATE             7090.0
                               GEOMETRIC_CENTER       37623.0
                               NO_GEOCODE                30.0
                               RANGE_INTERPOLATED     14137.0
                               ROOFTOP                75073.0
                               NaN                       11.0
address_state                                             3.0
                               São Paulo             133714.0
                               SP                       246.0
                               Santa Catarina             1.0
address_

##### Podemos ver que todas as observações dessa amostra são de imóveis em São Paulo. Consequentemente, podemos excluir as colunas *address_city*, *addres_state* e *address_country*. A variável *listingStatus* pode sair da análise por ser constante. Outras variáveis serão mantidas para análise de correlação.

In [ ]:
columns_to_remove += ['address_city', 'address_state', 'address_country', 'listingStatus']